# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 21 2022 1:40PM,242464,21,621590,"NBTY Global, Inc.",Released
1,Jul 21 2022 1:33PM,242463,10,SO94319,"Senseonics, Incorporated",Released
2,Jul 21 2022 1:25PM,242460,10,SOIVA0000499,Ivaoes Animal Health,Released
3,Jul 21 2022 1:25PM,242460,10,SOIVA0000500,Ivaoes Animal Health,Released
4,Jul 21 2022 1:25PM,242460,10,SOIVA0000501,Ivaoes Animal Health,Released
5,Jul 21 2022 1:25PM,242460,10,SOIVA0000502,Ivaoes Animal Health,Released
6,Jul 21 2022 1:25PM,242459,10,SOTAH0000735,"Nextsource Biotechnology, LLC",Released
7,Jul 21 2022 1:23PM,242458,10,8303568,Eywa Pharma Inc.,Released
8,Jul 21 2022 1:23PM,242458,10,8304699,Eywa Pharma Inc.,Released
9,Jul 21 2022 1:23PM,242458,10,8304701,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,242458,Released,3
43,242459,Released,1
44,242460,Released,4
45,242463,Released,1
46,242464,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242458,NaN,NaN,3.0
242459,NaN,NaN,1.0
242460,NaN,NaN,4.0
242463,NaN,NaN,1.0
242464,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242348,0.0,44.0,34.0
242365,16.0,6.0,0.0
242375,0.0,0.0,1.0
242376,0.0,0.0,1.0
242377,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242348,0,44,34
242365,16,6,0
242375,0,0,1
242376,0,0,1
242377,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242348,0,44,34
1,242365,16,6,0
2,242375,0,0,1
3,242376,0,0,1
4,242377,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242348,,44,34
1,242365,16,6,
2,242375,,,1
3,242376,,,1
4,242377,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 21 2022 1:40PM,242464,21,"NBTY Global, Inc."
1,Jul 21 2022 1:33PM,242463,10,"Senseonics, Incorporated"
2,Jul 21 2022 1:25PM,242460,10,Ivaoes Animal Health
6,Jul 21 2022 1:25PM,242459,10,"Nextsource Biotechnology, LLC"
7,Jul 21 2022 1:23PM,242458,10,Eywa Pharma Inc.
10,Jul 21 2022 12:56PM,242455,10,Yusen – 3D Matrix
11,Jul 21 2022 12:53PM,242452,19,ACG North America LLC
12,Jul 21 2022 12:49PM,242449,10,Yusen – 3D Matrix
13,Jul 21 2022 12:47PM,242448,10,"ClearSpec, LLC"
14,Jul 21 2022 12:42PM,242447,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 21 2022 1:40PM,242464,21,"NBTY Global, Inc.",,,1
1,Jul 21 2022 1:33PM,242463,10,"Senseonics, Incorporated",,,1
2,Jul 21 2022 1:25PM,242460,10,Ivaoes Animal Health,,,4
3,Jul 21 2022 1:25PM,242459,10,"Nextsource Biotechnology, LLC",,,1
4,Jul 21 2022 1:23PM,242458,10,Eywa Pharma Inc.,,,3
5,Jul 21 2022 12:56PM,242455,10,Yusen – 3D Matrix,,,1
6,Jul 21 2022 12:53PM,242452,19,ACG North America LLC,,,1
7,Jul 21 2022 12:49PM,242449,10,Yusen – 3D Matrix,,,1
8,Jul 21 2022 12:47PM,242448,10,"ClearSpec, LLC",,,1
9,Jul 21 2022 12:42PM,242447,10,Yusen – 3D Matrix,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 1:40PM,242464,21,"NBTY Global, Inc.",1,,
1,Jul 21 2022 1:33PM,242463,10,"Senseonics, Incorporated",1,,
2,Jul 21 2022 1:25PM,242460,10,Ivaoes Animal Health,4,,
3,Jul 21 2022 1:25PM,242459,10,"Nextsource Biotechnology, LLC",1,,
4,Jul 21 2022 1:23PM,242458,10,Eywa Pharma Inc.,3,,
5,Jul 21 2022 12:56PM,242455,10,Yusen – 3D Matrix,1,,
6,Jul 21 2022 12:53PM,242452,19,ACG North America LLC,1,,
7,Jul 21 2022 12:49PM,242449,10,Yusen – 3D Matrix,1,,
8,Jul 21 2022 12:47PM,242448,10,"ClearSpec, LLC",1,,
9,Jul 21 2022 12:42PM,242447,10,Yusen – 3D Matrix,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 1:40PM,242464,21,"NBTY Global, Inc.",1,,
1,Jul 21 2022 1:33PM,242463,10,"Senseonics, Incorporated",1,,
2,Jul 21 2022 1:25PM,242460,10,Ivaoes Animal Health,4,,
3,Jul 21 2022 1:25PM,242459,10,"Nextsource Biotechnology, LLC",1,,
4,Jul 21 2022 1:23PM,242458,10,Eywa Pharma Inc.,3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 1:40PM,242464,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jul 21 2022 1:33PM,242463,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Jul 21 2022 1:25PM,242460,10,Ivaoes Animal Health,4.0,NaN,NaN
3,Jul 21 2022 1:25PM,242459,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
4,Jul 21 2022 1:23PM,242458,10,Eywa Pharma Inc.,3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 1:40PM,242464,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jul 21 2022 1:33PM,242463,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Jul 21 2022 1:25PM,242460,10,Ivaoes Animal Health,4.0,0.0,0.0
3,Jul 21 2022 1:25PM,242459,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
4,Jul 21 2022 1:23PM,242458,10,Eywa Pharma Inc.,3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4363790,58.0,26.0,16.0
12,727333,1.0,2.0,0.0
15,727271,6.0,0.0,14.0
16,1212032,8.0,0.0,0.0
19,727297,3.0,0.0,0.0
20,484787,47.0,44.0,0.0
21,1696828,5.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4363790,58.0,26.0,16.0
1,12,727333,1.0,2.0,0.0
2,15,727271,6.0,0.0,14.0
3,16,1212032,8.0,0.0,0.0
4,19,727297,3.0,0.0,0.0
5,20,484787,47.0,44.0,0.0
6,21,1696828,5.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,58.0,26.0,16.0
1,12,1.0,2.0,0.0
2,15,6.0,0.0,14.0
3,16,8.0,0.0,0.0
4,19,3.0,0.0,0.0
5,20,47.0,44.0,0.0
6,21,5.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,58.0
1,12,Released,1.0
2,15,Released,6.0
3,16,Released,8.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,16.0,0.0,14.0,0.0,0.0,0.0,0.0
Executing,26.0,2.0,0.0,0.0,0.0,44.0,2.0
Released,58.0,1.0,6.0,8.0,3.0,47.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,16.0,0.0,14.0,0.0,0.0,0.0,0.0
1,Executing,26.0,2.0,0.0,0.0,0.0,44.0,2.0
2,Released,58.0,1.0,6.0,8.0,3.0,47.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,16.0,0.0,14.0,0.0,0.0,0.0,0.0
1,Executing,26.0,2.0,0.0,0.0,0.0,44.0,2.0
2,Released,58.0,1.0,6.0,8.0,3.0,47.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()